In [1]:
!pip install pyspark==3.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317123 sha256=10e18f6c0edb9df00849c9fe7503a0bea856ffd6d3154143cb072154deee9def
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [3]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [4]:
!wget -O police-stations.csv https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
!ls -l

--2023-09-04 21:42:47--  https://data.cityofchicago.org/api/views/z8bn-74gv/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.68.26, 52.206.140.205
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘police-stations.csv’

police-stations.csv     [ <=>                ]   5.57K  --.-KB/s    in 0s      

2023-09-04 21:42:47 (1.75 GB/s) - ‘police-stations.csv’ saved [5699]

total 12
-rw-r--r-- 1 root root 5699 Aug 19  2019 police-stations.csv
drwxr-xr-x 1 root root 4096 Aug 31 13:25 sample_data


In [5]:
ps = sc.textFile('police-stations.csv')
ps.first()

'DISTRICT,DISTRICT NAME,ADDRESS,CITY,STATE,ZIP,WEBSITE,PHONE,FAX,TTY,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION'

In [6]:
ps_header = ps.first()

In [8]:
ps_data = ps.filter(lambda line : line != ps_header)
ps_data.first()

'Headquarters,Headquarters,3510 S Michigan Ave,Chicago,IL,60653,http://home.chicagopolice.org,,,,1177731.401,1881697.404,41.83070169,-87.62339535,"(41.8307016873, -87.6233953459)"'

In [10]:
# how many police station
ps_data.map(lambda line: line.split(',')).count()

24

In [14]:
# select district ID, name, Address, and Zip code for the police stations with district id 7
(ps_data.filter(lambda line : line.split(',')[0] == '7').
 map(lambda line : (
      line.split(',')[0],
      line.split(',')[1],
      line.split(',')[2],
      line.split(',')[5],
 ) ).collect())

[('7', 'Englewood', '1438 W 63rd St', '60636')]

In [15]:
# select district ID, name, Address, and Zip code for the police stations with district id 10 and 11

(ps_data.filter(lambda line : line.split(',')[0] in ['10', '11']).
 map(lambda line : (
      line.split(',')[0],
      line.split(',')[1],
      line.split(',')[2],
      line.split(',')[5],
 ) ).collect())

[('10', 'Ogden', '3315 W Ogden Ave', '60623'),
 ('11', 'Harrison', '3151 W Harrison St', '60612')]